## Example Data Collection code: 
#### Obtaining  WEATHER data for certain date intervals and future forecasts.
#### Just obtain API keys for yourself and get your own data for the datetime interval you desired. 

## Geçmiş hava durumu

In [18]:
def get_historical_weather_data(api,city_name,city_id,start_date):
    import json
    import requests
    date_now=dt.datetime.now().strftime("%Y-%m-%d:%H")
    url="https://api.weatherbit.io/v2.0/history/hourly?city_id="+city_id+"&start_date="+start_date+"&end_date="+date_now+"&tz=local&key="+api
    response = requests.get(url)
    response.text
    d = json.loads(response.text)
    print(url) #take 2 hours more based on api location and Istanbul time
    datetime = [i["datetime"] for i in d["data"]]
    humidity = [i["rh"] for i in d["data"]]
    temp = [i["temp"] for i in d["data"]]
    data=pd.DataFrame()
    data["Tarih"]=datetime
    data[city_name+"_temp"]=temp
    data[city_name+"_humidity"]=humidity
    return data.iloc[2:]  # take 2 forward based on api location and Istanbul time

In [20]:
api = "22d709aca2744ebeba3645b5e636edd1" #optain your own API KEY and use it within its lifetime
start_date="2019-01-01"
IstanbulAnadolu = get_historical_weather_data(api,"Istanbul_anadolu",'745044',start_date)
IstanbulAvrupa  = get_historical_weather_data(api,"Istanbul_avrupa",'745044',start_date)
Bursa           = get_historical_weather_data(api,"Bursa",'750269',start_date)
historical_weather_orj = pd.concat([IstanbulAnadolu,IstanbulAvrupa,Bursa],axis=1)

historical_weather=historical_weather_orj.copy()
historical_weather["Tarih"]=historical_weather.pop("Tarih").iloc[:,1]

historical_weather.Tarih=historical_weather.Tarih.apply(lambda x: pd.to_datetime(x[:10]+" "+str(x[-2:])+ ":00"))


In [21]:
historical_weather

### Gelecek Hava tahmini

In [37]:
import requests
import pandas as pd
import requests
import json
import numpy as np
import os
import sqlite3
def get_forecast_weather_data(api,city_name,city_id,forecast):
    url="https://api.weatherbit.io/v2.0/forecast/hourly?city_id="+city_id+"&key="+api+"&hours="+str(forecast+2)
    print(url) #take 2 hours more based on api location and Istanbul time
    response = requests.get(url)
    d = json.loads(response.text)
    datetime = [i["datetime"] for i in d["data"]]
    humidity = [i["rh"] for i in d["data"]]
    temp = [i["temp"] for i in d["data"]]

    data=pd.DataFrame()
    data["Tarih"]=datetime
    data[city_name+"_temp"]=temp
    data[city_name+"_humidity"]=humidity
    return data.iloc[2:]  # take 2 forward based on api location and Istanbul time


In [38]:

api = "29385959e9174a32a43ef841beca77qw"

IstanbulAnadolu = get_forecast_weather_data(api,"Istanbul_anadolu",'745044',24)
IstanbulAvrupa  = get_forecast_weather_data(api,"Istanbul_avrupa",'745044',24)
Bursa           = get_forecast_weather_data(api,"Bursa",'750269',24)
forecast_weather_orj = pd.concat([IstanbulAnadolu,IstanbulAvrupa,Bursa],axis=1)

forecast_weather=forecast_weather_orj.copy()
forecast_weather["Tarih"]=forecast_weather.pop("Tarih").iloc[:,1]

forecast_weather.Tarih=forecast_weather.Tarih.apply(lambda x: pd.to_datetime(x[:10]+" "+str(x[-2:])+ ":00"))


In [39]:
historical_weather

In [40]:
forecast_weather

In [41]:
final_data_ind = pd.merge(df_merged,historical_weather,how='inner',on='Tarih')


In [42]:
final_data_ind

In [43]:
final_data_ind.to_csv("smfdb_with_weather.csv")